In [ ]:
import pandas as pd
import numpy as np
import math
import random
import multiprocessing as mp

n_processes = 5  # Number of runs you want to parallelize

def simulation(run_id):
  random.seed(run_id)
  np.random.seed(run_id)

  # Define data structures
  input = 'initial_conditions_201810.csv'
  output = f'output_data_run_{run_id}.csv'
  data = [] #a big table of results

  ### parameters that will be varied--------------------------------------------
  r1list = (0.1, .2, .3,.4,.5,.6,.7,.8,.9,1, 1.1, 1.2, 1.3, 1.4, 1.5) #varying r1
  s1list = (0.1, .2, .3,.4,.5,.6,.7,.8,.9,1, 1.1, 1.2, 1.3, 1.4, 1.5) #varying s1
  b1list = (0, 0.1, .2, .3,.4,.5,.6,.7,.8,.9, 1) #varying b1
  s0list = (0.03, 0.10, 1) # base rate of random entry. hiring parameters, s0=1 corresponds to simulating isolated firms
  r0list = (0.01, 0.05) # corresponding to high/low industry turnover conditions

  ### parameters that will be fixed---------------------------------------------
  # Set global parameters for simulations
  employee = 30 # number of employees per firm
  fm_no = 30 # number of firms
  time = 120 # number of time periods (months)
  var_win = 0.1 #which within variance in the input data?

  # departure parameters
  r0=0.03 # turnover base rate (3.5% monthly according to JOLTS)
  r2=0.05 # max increase in turnover probability

  # socialization parameters
  b0=0  # base rate of socialization
  b2=.3 # speed of socialization susceptibility decline by tenure
  b3=.1 # speed of socialization susceptibility decline by employments

  # Load the entire CSV file----------------------------------------------------
  df_all = pd.read_csv(input)
  filtered_df = df_all[df_all['var_win'] == var_win] # Filter rows
  num_units = len(filtered_df) // int(employee * fm_no) # Enumerate initial conditions

  # Loop over different initial conditions--------------------------------------
  for unit in range(num_units):

    start_row = unit * int(employee * fm_no)
    end_row = (unit + 1) * int(employee * fm_no)
    df_unit = filtered_df.iloc[start_row:end_row]

    # Extract updating variables from selected initial condition
    culture = list(df_unit['culture'])
    tenure = list(df_unit['tenure'])
    employments = list(df_unit['employments'])

  # Loop over different parameters ---------------------------------------------
    for s0 in s0list:
      for r0 in r0list:
        for r1 in r1list:
          for b1 in b1list:
            for s1 in s1list:

              ### REORG VARIABLES
              # c_all, t_all, e_all (or e_all copy): three 30*30 list of lists (~tables)
              # row is for a firm
              # column is for an employee in a firm

              # employees' cultural score
              c_all=[]
              for i in range(fm_no):
                c_firm = culture[i*employee: (i+1)*employee]
                c_all.append(c_firm)

              # tenure
              t_all=[]
              for i in range(fm_no):
                t_firm = tenure[i*employee: (i+1)*employee]
                t_all.append(t_firm)

            # prior employments
              e_all=[]
              for i in range(fm_no):
                e_firm = employments[i*employee: (i+1)*employee]
                e_all.append(e_firm)
              e_all_copy = e_all.copy()# used for recording update

              # initialize additional variables
              ave_hire_list=[]
              carrier_list=[]

              ### CALCULATIONS

              # initial firm culture as median of employees’ cultural scores
              firm_culture =[]
              for i in range(fm_no):
                firm_culture.append(np.median(c_all[i]))

              # initial between-variance
              betw_initial=0
              for i in range(fm_no):
                betw_initial = betw_initial + ((firm_culture[i] - np.mean(firm_culture))**2)
              betw_initial= (betw_initial/(fm_no-1))** (0.5)

              # initial within-variance
              single_fm_list1 =[]
              for i in range(fm_no):
                single_fm1=0
                for j in range(employee):
                  single_fm1 = single_fm1 +((c_all[i][j] -  firm_culture[i])**2)/(employee -1)
                single_fm_list1.append(single_fm1 **(0.5))
              within_initial = np.mean(single_fm_list1)

  #------------------loop over months-------------------------------------------
              for t in range(time):

                # update firm culture, always, for every round of simulation
                # this impacts all following processes
                for i in range(fm_no):
                  firm_culture[i] = np.median(c_all[i])

  #-----------departure---------------------------------------------------------
                # calculate departure probability for all employees
                dps =[] # list of lists for departure probabilities
                for i in range(fm_no):
                  dp =[] #departure probability for every step, every person in a single firm
                  for j in range(employee):
                    dp.append(random.random() > ((r0+r2) - r2* math.exp(-(((firm_culture[i] - c_all[i][j])**2)/(2* r1*r1)))))
                  dps.append(dp)

                # update tenure for all (assuming retained)
                for i in range(fm_no):
                  for j in range(employee):
                    if(dps[i][j] == True):
                      t_all[i][j] = t_all[i][j]+1
                      # do not update employments yet because you don't know if she will be rehired or not

                # determine who will leave based on pool of available indexes and their culture scores for each firm
                index0=[]# to track persons (index) who will leave
                available=[] #available pool of (departure) culture scores for each firm
                for i in range(fm_no):
                  index_temp=[]
                  ava_temp=[]
                  for j in range(employee):
                    if (dps[i][j] == False):
                      index_temp.append(j)
                      ava_temp.append(c_all[i][j])
                  index0.append(index_temp)
                  available.append(ava_temp)

                #remove departed employees from the firm
                for i in range(fm_no):
                  c_all[i] = list(np.delete(c_all[i], index0[i]))
                  t_all[i] = list(np.delete(t_all[i], index0[i]))
                  e_all_copy[i] = list(np.delete(e_all_copy[i], index0[i]))

  #-----------hiring------------------------------------------------------------
                # initialize average mobility
                carriers=0

                # calculate open spots in each firm
                count=[]
                for i in range(fm_no):
                  count.append(len(available[i]))
                ave_hire = np.mean(count)/employee # track average openings per firm

                # create slot-based random order for hiring
                random_order = [] # list of lists
                for i in range(len(count)):
                  repeat = [[i] * count[i]]
                  random_order += repeat

                overall=[] # convert list of lists to list
                for i in range(len(random_order)):
                  for j in range(len(random_order[i])):
                    overall.append(random_order[i][j])
                random.shuffle(overall)

                # loop over random order of slots
                for q in range(len(overall)):
                  i = overall[q]
                  other=[]

                  # you only consider hiring an employee from other firms
                  for k in range(fm_no):
                    if ((k == i) ==False):
                      other.append(available[k][:])

                  # reorg other
                  for u in range(1, len(other)):
                    other[0] = np.append(other[0], other[u])
                  other= (list(other[0])).copy()

                  # consider candidates that are within the acceptable cultural score range
                  others_c=[]
                  for n in range(len(other)):
                    tem = abs((firm_culture[i]) - (other[n]))
                    if (tem < 2*s1):
                      others_c.append(other[n])

  #----random entry-------------------------------------------------------------
                  if((random.random() < s0) or (len(others_c) ==0) ):
                    new = np.random.normal(firm_culture[i], s1, 1)
                    t_all[i] = list(np.append(t_all[i] , [0]))
                    c_all[i] = list(np.append(c_all[i] , new))
                    e_all_copy[i]  = list(np.append(e_all_copy[i] , [1]))

  #----hiring from available pool-----------------------------------------------
                  else:
                    # at least one exists
                    carriers= carriers+1  # track carriers
                    v=list(random.sample(others_c, 1))
                    c_all[i] =  list(np.append(c_all[i] , v))
                    t_all[i] = list( np.append(t_all[i] , [0]))
                    for a in range(fm_no):
                      for b in range(len(available[a])):
                        if (v[0] == available[a][b]):
                          value =index0[a][b]
                          e_all_copy[i] = list( np.append(e_all_copy[i] , [e_all[a][value]+1])) # use copy() so don't overwrite e_all

  #----socialization------------------------------------------------------------
                for i in range(fm_no):
                  for j in range(len(c_all[i])):
                    c_all[i][j] = c_all[i][j] + ( firm_culture[i] - c_all[i][j] )* (b1* math.exp(- b2* (t_all[i][j] -1) -b3* (e_all_copy[i][j] -1)) +b0)

                # after updating employments, save e_all and copy() for next period
                e_all = e_all_copy.copy() # don't overwrite
                e_all_copy = e_all.copy()

                # store average hires and carries per time period
                ave_hire_list.append(ave_hire)
                carrier_list.append(carriers/(employee*fm_no))

  #----end of time period-------------------------------------------------------
              ### CALCULATIONS AT END OF SIMULATION
              #within-variance (after)
              single_fm_list =[]
              for i in range(fm_no):
                single_fm=0
                for j in range(employee):
                  single_fm = single_fm +((c_all[i][j] - firm_culture[i] )**2)/(employee -1)
                single_fm_list.append(single_fm **(0.5))
              within = np.mean(single_fm_list)

              #betweenness (after)
              betw=0
              for i in range(fm_no):
                betw = betw + (firm_culture[i] - np.mean(firm_culture))**2
              betw= ((betw/(fm_no-1))** (0.5) )

              # other variables (after)
              tenure_median = np.median([item for sublist in t_all for item in sublist]) #median!
              employment_median = np.median([item for sublist in e_all for item in sublist])#median!
              turnover_average =np.mean(ave_hire_list)
              carriers_average = np.mean(carrier_list)

  #----keep data ---------------------------------------------------------------
              data.append({
                              'initcond': unit,
                              's0': s0,
                              'r0': r0,
                              'r1': r1,
                              'b1': b1,
                              's1': s1,
                              'betw_initial': betw_initial,
                              'betw_final': betw,
                              'within_initial': within_initial,
                              'within_final': within,
                              'tenure': tenure_median,
                              'employment': employment_median,
                              'turnover': turnover_average,
                              'carriers': carriers_average
                          })
  df = pd.DataFrame(data)
  df.to_csv(output, index=False)
  return f"Run {run_id} completed."

### parallelize-----------------------------------------------------------------
def main():
    with mp.Pool(n_processes) as pool:
        results = pool.map(simulation, range(n_processes))
    print(results)

if __name__ == '__main__':
    main()